In [1]:
import json
import os
from neo4j import GraphDatabase

# Load config
with open("healthcare_config.json", "r") as f:
    config = json.load(f)

os.environ["GOOGLE_API_KEY"] = config.get("google_api_key", "")

# Neo4j Connection
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    def run_query(self, query, params=None):
        with self.driver.session() as session:
            result = session.run(query, params or {})
            return [r.data() for r in result]

conn = Neo4jConnection(config["neo4j_uri"], config["neo4j_user"], config["neo4j_password"])
print("Connected!")

Connected!


## 1. Define Tool Functions

In [2]:
# Bed Management
def get_available_beds(department: str = None) -> str:
    """Get available beds by department."""
    if department:
        q = "MATCH (d:Department {name: $dept})-[:HAS_BED]->(b:Bed {status: 'available'}) RETURN d.name as dept, count(b) as beds"
        return json.dumps(conn.run_query(q, {"dept": department}))
    q = "MATCH (d:Department)-[:HAS_BED]->(b:Bed {status: 'available'}) RETURN d.name as dept, count(b) as beds ORDER BY d.name"
    return json.dumps(conn.run_query(q))

def get_icu_status() -> str:
    """Get ICU bed availability."""
    q = "MATCH (d:Department {name: 'ICU'})-[:HAS_BED]->(b:Bed) RETURN b.status as status, count(b) as count"
    return json.dumps(conn.run_query(q))

# Patient Management
def get_discharge_candidates() -> str:
    """Get patients ready for discharge."""
    q = "MATCH (p:Patient)-[:ADMITTED_TO]->(b:Bed)<-[:HAS_BED]-(d:Department) WHERE p.status IN ['ready_for_discharge', 'recovering'] RETURN p.name, p.status, d.name as dept"
    return json.dumps(conn.run_query(q))

def get_critical_patients() -> str:
    """Get patients in critical condition."""
    q = "MATCH (p:Patient {status: 'critical'})-[:ADMITTED_TO]->(b:Bed)<-[:HAS_BED]-(d:Department) RETURN p.name, p.condition, d.name as dept"
    return json.dumps(conn.run_query(q))

# Inventory
def get_low_stock_items() -> str:
    """Get supplies running low."""
    q = "MATCH (d:Department)-[:STOCKS]->(i:InventoryItem) WHERE i.quantity < i.minStock RETURN i.name, i.quantity, i.minStock, d.name as dept"
    return json.dumps(conn.run_query(q))

# Physicians
def get_physicians_on_call() -> str:
    """Get physicians currently on call."""
    q = "MATCH (p:Physician {onCall: true})-[:ASSIGNED_TO]->(d:Department) RETURN p.name, p.specialty, d.name as dept"
    return json.dumps(conn.run_query(q))

# Custom Cypher
def run_cypher(query: str) -> str:
    """Run custom Cypher query."""
    try:
        return json.dumps(conn.run_query(query))
    except Exception as e:
        return json.dumps({"error": str(e)})

## 2. Create ADK Agents

In [3]:
from google.adk.agents import Agent
from google.adk.tools import FunctionTool

# Bed Management Agent
bed_agent = Agent(
    name="bed_management_agent",
    model="gemini-2.0-flash",
    description="Handles bed capacity and availability queries",
    instruction="You manage hospital bed information. Use tools to check availability.",
    tools=[FunctionTool(get_available_beds), FunctionTool(get_icu_status)]
)

# Patient Agent
patient_agent = Agent(
    name="patient_research_agent",
    model="gemini-2.0-flash",
    description="Handles patient data and discharge planning",
    instruction="You research patient information. Use tools for patient queries.",
    tools=[FunctionTool(get_discharge_candidates), FunctionTool(get_critical_patients)]
)

# Inventory Agent
inventory_agent = Agent(
    name="inventory_agent",
    model="gemini-2.0-flash",
    description="Handles medical supplies and equipment status",
    instruction="You manage inventory information. Check supply levels.",
    tools=[FunctionTool(get_low_stock_items)]
)

# Physician Agent
physician_agent = Agent(
    name="physician_agent",
    model="gemini-2.0-flash",
    description="Handles physician schedules and on-call status",
    instruction="You manage physician information and schedules.",
    tools=[FunctionTool(get_physicians_on_call)]
)

print("Sub-agents created!")

Sub-agents created!


## 3. Create Root Orchestrator

In [4]:
root_agent = Agent(
    name="healthcare_command_center",
    model="gemini-2.0-flash",
    description="Healthcare Command Center orchestrator",
    instruction="""You are the Healthcare Command Center assistant.
    Route queries to the appropriate sub-agent:
    - Bed questions -> bed_management_agent
    - Patient questions -> patient_research_agent
    - Supply/equipment -> inventory_agent
    - Physician/doctor -> physician_agent
    Provide clear, actionable responses.""",
    sub_agents=[bed_agent, patient_agent, inventory_agent, physician_agent]
)

print("Root orchestrator created!")

Root orchestrator created!


## 4. Sample Queries

In [17]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

session_service = InMemorySessionService()
runner = Runner(
    agent=root_agent,
    app_name="healthcare",
    session_service=session_service
)

# Create session
USER_ID = "admin"
session = await session_service.create_session(app_name="healthcare", user_id=USER_ID)
print(f"Session created: {session.id}")

async def ask(question: str):
    print(f"\n📋 Query: {question}")
    print("-" * 50)
    
    # Create Content object for the message
    message = types.Content(role="user", parts=[types.Part(text=question)])
    
    # run_async returns an async generator, iterate through events
    final_response = None
    async for event in runner.run_async(
        session_id=session.id, 
        user_id=USER_ID,
        new_message=message
    ):
        # Get the final response from events
        if hasattr(event, 'content') and event.content:
            if hasattr(event.content, 'parts'):
                for part in event.content.parts:
                    if hasattr(part, 'text') and part.text:
                        final_response = part.text
    
    print(f"Response: {final_response}")
    return final_response

Session created: 2052b3ef-da1e-4c77-afab-d0bf2b7e46b5


In [21]:
# Query 1: ICU Beds
await ask("How many ICU beds are available?")


📋 Query: How many ICU beds are available?
--------------------------------------------------


/Users/saritajoshi/testbedmanagement/.venv/lib/python3.13/site-packages/google/adk/flows/llm_flows/base_llm_flow.py:449: UserWarning: [EXPERIMENTAL] feature FeatureName.PROGRESSIVE_SSE_STREAMING is enabled.
  async for event in agen:
/Users/saritajoshi/testbedmanagement/.venv/lib/python3.13/site-packages/google/adk/flows/llm_flows/base_llm_flow.py:449: UserWarning: [EXPERIMENTAL] feature FeatureName.PROGRESSIVE_SSE_STREAMING is enabled.
  async for event in agen:


Response: There are 6 ICU beds available.


'There are 6 ICU beds available.'

In [ ]:
# Query 2: Discharge Planning
await ask("Which patients need discharge planning?")

In [ ]:
# Query 3: Low Supplies
await ask("What supplies are running low?")

In [ ]:
# Query 4: Physicians On Call
await ask("Which physicians are on call today?")

## 5. Direct Query Testing (Without ADK)

In [19]:
# Test tools directly without ADK
print("=== ICU Status ===")
print(get_icu_status())

print("\n=== Discharge Candidates ===")
print(get_discharge_candidates())

print("\n=== Low Stock Items ===")
print(get_low_stock_items())

print("\n=== Physicians On Call ===")
print(get_physicians_on_call())

=== ICU Status ===
[{"status": "occupied", "count": 9}, {"status": "maintenance", "count": 5}, {"status": "available", "count": 6}]

=== Discharge Candidates ===
[{"p.name": "Patient 4", "p.status": "ready_for_discharge", "dept": "ICU"}, {"p.name": "Patient 6", "p.status": "ready_for_discharge", "dept": "ICU"}, {"p.name": "Patient 8", "p.status": "ready_for_discharge", "dept": "ICU"}, {"p.name": "Patient 9", "p.status": "recovering", "dept": "ICU"}, {"p.name": "Patient 10", "p.status": "recovering", "dept": "Emergency"}, {"p.name": "Patient 11", "p.status": "recovering", "dept": "Emergency"}, {"p.name": "Patient 13", "p.status": "recovering", "dept": "Emergency"}, {"p.name": "Patient 15", "p.status": "recovering", "dept": "Emergency"}, {"p.name": "Patient 16", "p.status": "recovering", "dept": "Emergency"}, {"p.name": "Patient 17", "p.status": "recovering", "dept": "Emergency"}, {"p.name": "Patient 18", "p.status": "recovering", "dept": "Emergency"}, {"p.name": "Patient 19", "p.status"

## 6. Summary

This Healthcare Command Center demonstrates:
1. **Neo4j Graph Database** with healthcare schema
2. **Specialized Sub-Agents** for specific domains
3. **Root Orchestrator** for query routing
4. **Natural Language Interface** to hospital data

### Notebooks in Series:
- `09a_healthcare_graphrag_setup.ipynb` - Setup & Schema
- `09b_healthcare_data_population.ipynb` - Data Population
- `09c_healthcare_agents.ipynb` - Agent Definitions
- `09d_healthcare_orchestrator.ipynb` - Orchestrator & Queries